In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Toggle Code"></form>''')

In [2]:
%%capture
# Prepare
from plotly.offline import init_notebook_mode, iplot, plot
init_notebook_mode(connected=True)
import pandas as pd
import numpy as np
import os, glob
from plotly.graph_objs import *
os.chdir('/Users/denis/Documents/Projects/cmap/cmap-l1000-analysis')
%load_ext rpy2.ipython
%R source('/Users/denis/Documents/Projects/scripts/Support.R')

# Activate Plotly - hacky way to fix an incomprehensible bug
x = np.random.randn(2000)
y = np.random.randn(2000)
iplot([Histogram2dContour(x=x, y=y, contours=Contours(coloring='heatmap')),
       Scatter(x=x, y=y, mode='markers', marker=Marker(color='white', size=3, opacity=0.3))], show_link=False)

In [3]:
%%R
### Load data
# Get infiles
infiles <- c('f2-pca.dir/cmap_l1000-pca.rda',
             'f2-pca.dir/cmap_l1000-celltype_pca.rda',
             'f1-processed_data.dir/cmap_l1000-sample_annotations.txt')

# Load infiles
load(infiles[1])
load(infiles[2])
sampleAnnotationDataframe <- read.table(infiles[3], sep='\t', header=TRUE, row.names='sample_id')

### Prepare annotation dataframe
# Read sample annotation dataframe
sampleAnnotationDataframe <- read.table(infiles[3], sep='\t', header=TRUE, row.names='sample_id')

# Fix annotation dataframe
sampleAnnotationDataframe <- sampleAnnotationDataframe[,c('cell_id', 'pert_iname', 'pert_dose','pert_2_iname', 'pert_2_dose')]
sampleAnnotationDataframe$sample_id <- rownames(sampleAnnotationDataframe)
rownames(sampleAnnotationDataframe) <- gsub(':', '.', rownames(sampleAnnotationDataframe))

# Add plate names
sampleAnnotationDataframe$plateName <- sapply(sampleAnnotationDataframe$sample_id, function(x) strsplit(gsub('.','_',x,fixed=TRUE), '_')[[1]][4])

# Add perturbation 1
sampleAnnotationDataframe$perturbation_1 <- paste0(sampleAnnotationDataframe$pert_iname, ' (', round(sampleAnnotationDataframe$pert_dose, digits=1), ' um)')
sampleAnnotationDataframe$perturbation_1 <- gsub(' (-666 um)', '', sampleAnnotationDataframe$perturbation_1, fixed=TRUE)

# Add perturbation 2
sampleAnnotationDataframe$perturbation_2 <- paste0(sampleAnnotationDataframe$pert_2_iname, ' (', round(sampleAnnotationDataframe$pert_2_dose, digits=1), ' um)')
sampleAnnotationDataframe$perturbation_2 <- gsub('-666 (-666 um)', '', sampleAnnotationDataframe$perturbation_2, fixed=TRUE)

# Merge perturbations
sampleAnnotationDataframe$perturbationLabel <- paste(sampleAnnotationDataframe$perturbation_1, sampleAnnotationDataframe$perturbation_2, sep='<br>')

### Prepare PCA dataframe
# Get PCs
PCs <- c('PC1', 'PC2', 'PC3')

# Get plot dataframe
pcaDataframe <- as.data.frame(pca$x[,PCs])

# Add cell type
pcaDataframe$cellType <- sampleAnnotationDataframe[, 'cell_id'] # det_plate # cell_id # pert_id

# Merge with sample annotations
pcaDataframe <- merge(pcaDataframe, sampleAnnotationDataframe, by='row.names')[,c('Row.names','PC1','PC2','PC3','cellType','perturbationLabel')]

# Get variance labels
varLabels <- pca$varLabels

### Prepare cell-type PCA dataframe list
# Get cell types
cellTypes <- names(pcaResults)

# Prepare dataframes
celltypePcaDataframes <- sapply(cellTypes, function(x){ resultDataframe <- as.data.frame(pcaResults[[x]]$x[,PCs]);
                                                        resultDataframe$plate_name <- sapply(rownames(resultDataframe), function(x) strsplit(gsub('.','_',x,fixed=TRUE), '_')[[1]][4]);
                                                        resultDataframe <- merge(resultDataframe, sampleAnnotationDataframe, by.x='row.names', by.y="sample_id")
                                                        return(resultDataframe);}, simplify=FALSE)
                                                       
# Prepare variance labels
celltypeVarLabels <- sapply(cellTypes, function(x) pcaResults[[x]]$varLabels)

In [4]:
# Plot initial scatter
%Rpull PCs
%Rpull pcaDataframe
%Rpull varLabels
varLabelDict = {x.split(' ')[0]:x for x in varLabels}
cellTypes = list(set(pcaDataframe['cellType']))

In [5]:
def plot3dScatter(dataframe, categoricalColumn, varLabelDict, annotationColumn=False, title='', PCs=PCs, colors=['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00']):

    # Define empty plot
    p = []
    
    # Get unique categories
    uniqueCategories = list(set(dataframe[categoricalColumn]))

    # Get dict with unique categories
    categoricalExpressionDict = {x:dataframe[dataframe[categoricalColumn] == x] for x in uniqueCategories}

    # Loop through categories
    for i in range(len(uniqueCategories)):

        # Get category
        category = uniqueCategories[i]
    
        # Get plot dataframe
        plotDataframe = categoricalExpressionDict[category]
        
        # Get annotation, if specified
        annotation = plotDataframe[annotationColumn] if annotationColumn else ''

        # Append trace
        p.append(
            Scatter3d(
                x = plotDataframe[PCs[0]],
                y = plotDataframe[PCs[1]],
                z = plotDataframe[PCs[2]],
                mode='markers',
                text=annotation,
                name=category,
                marker=dict(
                    size=5,
                    color=colors[i],
                    opacity=0.9
                ),
            )
        )
        
    # Add layout
    layout = Layout(
        title=title,
        hovermode='closest',
        width=1000,
        height=1000,
        scene=dict(
            xaxis=dict(title=varLabelDict[PCs[0]]),
            yaxis=dict(title=varLabelDict[PCs[1]]),
            zaxis=dict(title=varLabelDict[PCs[2]]),
        ),
        margin=dict(
            l=50,
            r=50,
            b=50,
            t=50
        )
    )
    
    # Prepare figure
    fig = Figure(data=p, layout=layout)
    
    # Return figure
    return fig

In [6]:
fig = plot3dScatter(pcaDataframe, 'cellType', varLabelDict, annotationColumn='perturbationLabel', title='PCA Analysis of CMAP L1000 data<br>grouped by cell line')
iplot(fig)

In [7]:
# Loop through cell types
for cellType in cellTypes:

    # Pull dataframe
    %R -i cellType
    %R celltypePcaDataframe <- celltypePcaDataframes[[cellType]]
    %Rpull celltypePcaDataframe

    # Pull variance labels
    %R celltypeVarLabel <- celltypeVarLabels[[cellType]]
    %Rpull celltypeVarLabel
    celltypeVarLabelDict = {x.split(' ')[0]:x for x in celltypeVarLabel}    

    # Plot
    fig = plot3dScatter(celltypePcaDataframe, 'plateName', celltypeVarLabelDict, annotationColumn='perturbationLabel', title='PCA Analysis of CMAP L1000 data<br>%(cellType)s cells, grouped by plate' % locals())
    iplot(fig)